## Context2Canvas - AI ggplot2 Code Generation

Brief Elevator Pitch of our project 

## Setup the functions for prompting

In [2]:
from openai import OpenAI
client = OpenAI()

def prompt_model(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[
            {"role": "user", 'content': prompt}
        ]
    )
    return completion.choices[0].message.content

## Chunk data and generate indices (setting up vector database)

User queries will be matched to indexes that best approximate the text chunks used to summarize an answer. For this assignment, you may chunk the text and then prompt the model to generate questions that are answerable by the text. The generated questions can then be used as the "documents" stored in the vector database.

In [3]:
#Load the data

import json
with open('annotations.json', 'r') as f:
    data = json.load(f)



In [6]:
#import langchain chunking library
from langchain_text_splitters import CharacterTextSplitter

#from langchain_text_splitters import RecursiveCharacterTextSplitter 
#we can use better parsing techniques in a future update


In [7]:
#set up text splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=2500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)


In [4]:
#chunk data
annotations = [str(record["general_figure_info"]) for record in data]
types = [record["type"] for record in data]

print(len(annotations) == len(types))

True


## Build the vector database

When building the vector database, be sure to maintain a mapping between the generated questions and the chunks that can be used later to retrieve the chunks from the most similar indices to the user query provided.

You may also add the function to query the vector database that you will use later.

In [5]:
#setting up the collection
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="c2c")



In [6]:
#add data to collection
collection.add(
    documents = annotations[:1000], 
    ids = [str(i) for i in range(len(annotations[:1000]))]
)

## Analyze Input Data

In [7]:
#collect user input data

import pandas as pd

#filepath = input("insert filepath of the dataset you want to analyze")
df = pd.read_csv("pixar_films.csv")

In [9]:
# what are the column names associated with this dataset?
columns = prompt_model(f"What are the variables associated with this dataset: {df}? Return them as a python dictionary, in the format of 'variable_name: data_type'. Do not return any dialogue besides the python dictionary.")

#generate summary statistics
summaryStats = prompt_model(f"using {df}, generate me summary statistics of each variable according to these {statStandards}. Do not return any dialogue besides the summary statistics.")

# Create Interesting data questions based on the variables + summary statistics
question = prompt_model(f"create an interesting data question based on {columns} and {summaryStats}. Do not return anything besides the data question.")

# gather visualization type from data question
vizType = prompt_model(f"what is the best visualization class we should use to characterize this problem, given {question}, {columns}, and {summaryStats}? Do not return anything besides the visualization type. Only return a type listed in {supportedClasses}")
print(vizType)



line-plot


## Use VectorDB to query similar examples for few-shot learning

In [11]:
type(question)

str

In [13]:
examples = collection.query(
    query_texts= [question], # Chroma will embed this for you
    n_results=3 # how many results to return
)
print(examples)

{'ids': [['600', '899', '117']], 'embeddings': None, 'documents': [["{'title': {'text': 'title', 'bbox': {'y': 9, 'x': 101.43156250000001, 'w': 50.234375, 'h': 16}}, 'x_axis': {'minor_ticks': {'values': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], 'bboxes': [{'y': 350, 'x': 108, 'w': 4, 'h': 4}, {'y': 350, 'x': 120, 'w': 4, 'h': 4}, {'y': 350, 'x': 133, 'w': 4, 'h': 4}, {'y': 350, 'x': 145, 'w': 4, 'h': 4}, {'y': 350, 'x': 157, 'w': 4, 'h': 4}, {'y': 350, 'x': 170, 'w': 4, 'h': 4}, {'y': 350, 'x': 182, 'w': 4, 'h': 4}, {'y': 350, 'x': 194, 'w': 4, 'h': 4}, {'y': 350, 'x': 207, 'w': 4, 'h': 4}, {'y': 350, 'x': 219, 'w': 4, 'h': 4}, {'y': 350, 'x': 232, 'w': 4, 'h': 4}, {'y': 350, 'x': 244, 'w': 4, 'h': 4}, {'y': 350, 'x': 256, 'w': 4, 'h': 4}, {'y': 350, 'x'

## Generate the graph's code based on the collected examples


In [ ]:
codeOutput = prompt_model(f"generate python code for this visualization, given {vizType}, {question}, {columns}, and {summaryStats}")
print(codeOutput)

In [ ]:
#python code with vector databse helper hints
codeOutput = prompt_model(f"generate python code for this visualization, given {vizType}, {question}, {columns}, and {summaryStats}. Model your output on the examples given by {examples}")
print(codeOutput)

## Analysis of C2C's Performance

In [ ]:
from bert_score import BERTScorer

scorer = BERTScorer(model_type='bert-base-uncased')